In [2]:
import psycopg2
import time
import boto3
import copy
import gc

In [4]:
db_connection = psycopg2.connect(
            user='USER_STRING_HERE', password='PW_STRING_HERE', # Credentials removed for Git
            host='mads-capstone.cmohac77hep9.eu-north-1.rds.amazonaws.com', port=5432,
            database="mads"
        )

db_connection.autocommit = True

cursor = db_connection.cursor()

In [4]:
#select_query = "SELECT count(*) FROM greeks"

#cursor.execute(select_query)
#rows = cursor.fetchall()
#print(rows)

In [6]:
#delete_query = "DROP TABLE greeks2"
#cursor.execute(delete_query)

In [5]:
import pandas as pd

bucket = 'mads-capstone-2023' 
data_key = 'greeks2.csv' 
greeks_csv = 's3://{}/{}'.format(bucket, data_key) 

greeks_df = pd.read_csv(greeks_csv, nrows=5) ## Reading only 5 rows to check dataset format.

greeks_df

,date,increment,value,ticker,greek,option_type
0,2014-01-15 00:00:00,360,0.4762,LXU,iv,call
1,2014-01-15 00:00:00,360,0.5087,LXU,iv,put
2,2014-01-15 00:00:00,360,0.4805,LXU,iv,all
3,2014-01-15 00:00:00,360,3702.6400,LXU,gamma,call
4,2014-01-15 00:00:00,360,-420.8000,LXU,gamma,put


In [7]:
query = '''CREATE TABLE greeks3 (id SERIAL PRIMARY KEY,  
                                exec_date date,
                                increment SMALLINT,
                                opt_value float,
                                ticker VARCHAR(8),
                                greek VARCHAR(16),
                                option_type VARCHAR(8))'''


cursor.execute(query)

In [74]:
greeks_csv

's3://mads-capstone-2023/greeks2.csv'

%%time

copy_query = "COPY greeks2 (exec_date, increment, opt_value, ticker, greek, option_type) FROM 's3://mads-capstone-2023/greeks2.csv' WITH CSV"

cursor.execute(long_query)

In [ ]:
%%time

bucket = 'mads-capstone-2023' 
data_key = 'greeks2.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

insert_query = "INSERT INTO greeks3 (exec_date, increment, opt_value, ticker, greek, option_type) values ('{}',{},{},'{}','{}','{}');"

long_query = ""

for i, row in enumerate(obj["Body"].iter_lines()):
    if i == 0:
        continue
    #if i<=18100000: ## First Run failed at 18.1 Mn records. Condition to skip records till last fail count.
    #    if (i%1_000_000) ==0:
    #        print(f"{i} Records Passed.")
    #    continue
    try:
        row_data = row.decode().split(",")
        dt, incr, opt_value, ticker, greek, opt_type = row_data
        dt = dt.split()[0]
        incr = int(incr) if incr else "NULL"
        opt_value = float(opt_value) if opt_value else "NULL"
    except Exception as e:
        print(e)
        print(f"{i} - {row_data} Failed Parsing.")
        continue
        
    query = insert_query.format(dt,incr,opt_value,ticker,greek,opt_type)
    long_query += copy.copy(query + "\n")
    #print(query)
    if (i%100_000)==0: ## Batching 100k records and uploading all at once.
        cursor.execute(long_query)
        long_query = ""
        gc.collect();
        if (i%1_000_000) == 0:
            print(f"{i} Records Inserted")
        

if long_query:
    cursor.execute(long_query)

In [29]:
import pandas as pd

select_query = "SELECT count(*) FROM greeks4"

cursor.execute(select_query)

cursor.fetchall()

[(3400000,)]

In [5]:
import pandas as pd

select_query = "SELECT * FROM greeks2"

cursor.execute(select_query)

rows = cursor.fetchall()
records = []
for row in rows:
    records.append([row[0], row[1],row[2], row[3], row[4], row[5], row[6]])
    
df = pd.DataFrame(records, columns=["ID", "DATE", "INCREMENT", "VALUE", "TICKER", "GREEK", "OPTION_TYPE"])

print(df.shape)

df

(560000, 7)


,ID,DATE,INCREMENT,VALUE,TICKER,GREEK,OPTION_TYPE
0,1,2016-08-05,90,-236788.1100,HSY,zomma,all
1,2,2016-08-05,90,-498.3700,HSY,speed,call
2,3,2016-08-05,90,1985.0000,HSY,speed,put
3,4,2016-08-05,90,1486.6300,HSY,speed,all
4,5,2016-08-05,90,-595016.0600,HSY,color,call
...,...,...,...,...,...,...,...
559995,559996,2016-08-09,0,0.5808,AMSC,iv,put
559996,559997,2016-08-09,0,0.5836,AMSC,iv,all
559997,559998,2016-08-09,0,654.7200,AMSC,delta,call
559998,559999,2016-08-09,0,6474.2000,AMSC,delta,put


In [45]:
cursor.execute("DROP TABLE greeks2;")

In [ ]:
db_connection.close()